# Wolfram Elementary Cellular Automata

This notebook implements [Wolfram's Elementary Cellular Automata](https://mathworld.wolfram.com/ElementaryCellularAutomaton.html) - one-dimensional cellular automata where each cell's next state depends on its current state and its two immediate neighbors.

With 3 cells determining the next state (left neighbor, self, right neighbor), there are 2³ = 8 possible input configurations. Each rule (0-255) specifies an output bit for each configuration, giving us 2⁸ = 256 possible rules.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Optional
from IPython.display import display

## Core Implementation

The `WolframCA` class encapsulates the cellular automaton logic.

In [ ]:
class WolframCA:
    """Elementary Cellular Automaton implementation."""
    
    def __init__(self, rule: int, width: int = 101):
        """
        Initialize a Wolfram CA.
        
        Args:
            rule: Rule number (0-255)
            width: Width of the CA grid
        """
        if not 0 <= rule <= 255:
            raise ValueError("Rule must be between 0 and 255")
        
        self.rule = rule
        self.width = width
        self.rule_table = self._build_rule_table(rule)
        self.history: list[np.ndarray] = []
    
    def _build_rule_table(self, rule: int) -> dict[tuple[int, int, int], int]:
        """Build lookup table for the rule."""
        table = {}
        # Each 3-bit neighborhood maps to an output bit
        # Neighborhoods: 111, 110, 101, 100, 011, 010, 001, 000
        for i in range(8):
            # Extract the i-th bit from the rule number
            output = (rule >> i) & 1
            # Convert i to a 3-bit neighborhood (left, center, right)
            left = (i >> 2) & 1
            center = (i >> 1) & 1
            right = i & 1
            table[(left, center, right)] = output
        return table
    
    def initialize(self, initial_state: Optional[np.ndarray] = None) -> np.ndarray:
        """
        Initialize the CA with a starting state.
        
        Args:
            initial_state: Custom initial state, or None for single center cell
        
        Returns:
            The initial state array
        """
        if initial_state is not None:
            state = initial_state.copy()
        else:
            # Default: single cell in the center
            state = np.zeros(self.width, dtype=np.uint8)
            state[self.width // 2] = 1
        
        self.history = [state]
        return state
    
    def step(self) -> np.ndarray:
        """Advance the CA by one generation."""
        if not self.history:
            self.initialize()
        
        current = self.history[-1]
        next_state = np.zeros_like(current)
        
        for i in range(self.width):
            # Wrap around at boundaries (periodic boundary conditions)
            left = current[(i - 1) % self.width]
            center = current[i]
            right = current[(i + 1) % self.width]
            
            next_state[i] = self.rule_table[(left, center, right)]
        
        self.history.append(next_state)
        return next_state
    
    def run(self, generations: int) -> np.ndarray:
        """
        Run the CA for a specified number of generations.
        
        Args:
            generations: Number of generations to simulate
        
        Returns:
            2D array of the complete history
        """
        if not self.history:
            self.initialize()
        
        for _ in range(generations):
            self.step()
        
        return np.array(self.history)
    
    def get_history(self) -> np.ndarray:
        """Return the full history as a 2D numpy array."""
        return np.array(self.history)
    
    def reset(self):
        """Clear the history."""
        self.history = []
    
    def __repr__(self) -> str:
        return f"WolframCA(rule={self.rule}, width={self.width})"

## Visualization

In [ ]:
def plot_ca(ca: WolframCA, figsize: tuple[int, int] = (12, 8), cmap: str = 'binary'):
    """
    Plot the CA evolution.
    
    Args:
        ca: The WolframCA instance to plot
        figsize: Figure size (width, height)
        cmap: Matplotlib colormap
    """
    history = ca.get_history()
    
    fig, ax = plt.subplots(figsize=figsize)
    ax.imshow(history, cmap=cmap, interpolation='nearest', aspect='auto')
    ax.set_title(f'Rule {ca.rule}', fontsize=14)
    ax.set_xlabel('Cell Position')
    ax.set_ylabel('Generation')
    plt.tight_layout()
    plt.show()


def plot_rule_table(rule: int):
    """Visualize the rule lookup table."""
    fig, axes = plt.subplots(1, 8, figsize=(14, 2))
    fig.suptitle(f'Rule {rule} Transition Table', fontsize=12)
    
    for i, ax in enumerate(axes):
        # Neighborhoods are ordered 7,6,5,4,3,2,1,0 (111 to 000)
        idx = 7 - i
        output = (rule >> idx) & 1
        
        # Create a 2x3 grid showing input neighborhood and output
        grid = np.zeros((2, 3))
        grid[0, 0] = (idx >> 2) & 1  # left
        grid[0, 1] = (idx >> 1) & 1  # center
        grid[0, 2] = idx & 1         # right
        grid[1, 1] = output          # output (center of bottom row)
        
        ax.imshow(grid, cmap='binary', vmin=0, vmax=1)
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_title(f'{idx:03b}→{output}', fontsize=10)
    
    plt.tight_layout()
    plt.show()

## Example: Rule 30

Rule 30 is famous for producing chaotic, aperiodic patterns from a simple initial condition. It has been used as a pseudorandom number generator.

In [ ]:
# Visualize Rule 30's transition table
plot_rule_table(30)

In [ ]:
# Run Rule 30
ca30 = WolframCA(rule=30, width=201)
ca30.initialize()
ca30.run(100)
plot_ca(ca30)

## Example: Rule 110

Rule 110 is proven to be Turing complete - it can simulate any computation given the right initial conditions.

In [ ]:
plot_rule_table(110)

In [ ]:
ca110 = WolframCA(rule=110, width=201)
ca110.initialize()
ca110.run(100)
plot_ca(ca110)

## Example: Rule 90

Rule 90 produces the Sierpiński triangle - a classic fractal pattern.

In [ ]:
plot_rule_table(90)

In [ ]:
ca90 = WolframCA(rule=90, width=201)
ca90.initialize()
ca90.run(100)
plot_ca(ca90)

## Interactive Exploration

Try different rules and initial conditions below.

In [ ]:
def explore_rule(rule: int, width: int = 201, generations: int = 100, 
                 random_init: bool = False, density: float = 0.5):
    """
    Explore a CA rule with customizable parameters.
    
    Args:
        rule: Rule number (0-255)
        width: Grid width
        generations: Number of generations to simulate
        random_init: If True, use random initial state
        density: Probability of each cell being 1 (if random_init=True)
    """
    ca = WolframCA(rule=rule, width=width)
    
    if random_init:
        initial = (np.random.random(width) < density).astype(np.uint8)
        ca.initialize(initial)
    else:
        ca.initialize()
    
    ca.run(generations)
    
    plot_rule_table(rule)
    plot_ca(ca)

In [ ]:
# Try Rule 184 (traffic flow model)
explore_rule(184, random_init=True, density=0.3)

In [ ]:
# Explore your own rule!
explore_rule(rule=73, width=201, generations=100)

## Rule Classification

Wolfram classified CA rules into four classes:

1. **Class I**: Evolution leads to homogeneous state (e.g., Rule 0, 32)
2. **Class II**: Evolution leads to periodic structures (e.g., Rule 4, 108)
3. **Class III**: Evolution leads to chaotic/aperiodic patterns (e.g., Rule 30, 45)
4. **Class IV**: Complex localized structures, long transients (e.g., Rule 110, 54)

In [ ]:
# Compare rules from different classes
class_examples = {
    'Class I (Homogeneous)': 32,
    'Class II (Periodic)': 4,
    'Class III (Chaotic)': 30,
    'Class IV (Complex)': 110
}

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

for ax, (class_name, rule) in zip(axes.flat, class_examples.items()):
    ca = WolframCA(rule=rule, width=151)
    ca.initialize()
    ca.run(75)
    
    ax.imshow(ca.get_history(), cmap='binary', interpolation='nearest', aspect='auto')
    ax.set_title(f'{class_name}: Rule {rule}', fontsize=11)
    ax.set_xlabel('Cell Position')
    ax.set_ylabel('Generation')

plt.tight_layout()
plt.show()